# Get Simpsons episodes scripts

We want to gather a large dataset of Simpons transcripts to analyze and model. This notebook will show the steps to scraping text from HTML and putting it into a PostgreSQL database.

In [3]:
# imports
import bs4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import urllib
import time
sns.set()
%matplotlib inline

## Extract links from website

We'll start with [www.simpsonsworld.com](www.simpsonsworld.com), where many episodes and extra features can be found.

In [4]:
# set url
url = 'http://www.simpsonsworld.com/browse/episodes'

# read url
html_doc = urllib.request.urlopen(url).read()

# create soup object
soup = bs4.BeautifulSoup(html_doc, 'html.parser')

### Getting episode links

We can get all the links for each episode as a list by writing a for loop:

`episodes = []
for link in soup.find_all('div', attrs={'class': 'share-button'}):
    episodes.append(link.get('data-video-link'))`

This is equivalent to:

`ep_url = [link.get('data-video-link') for link in soup.find_all('div', attrs={'class': 'share-button'})]`

Since we also want other details about each episode, we'll build our list of lists using a for loop.

In [5]:
# create list of simpsons episodes
simpsonslist = list()

for instance in soup.find_all('div', attrs={'class': 'share-button'}):
    ep_sea = instance.get('data-season-number')
    ep_num = instance.get('data-episode-number')
    ep_nam = instance.get('data-video-name')
    ep_des = instance.get('data-video-description')
    ep_url = instance.get('data-video-link')
    simpsonslist.append([ep_sea, ep_num, ep_nam, ep_des, ep_url])

headings = ['season', 'number', 'name', 'description', 'url']

# create pandas dataframe
df1 = pd.DataFrame(simpsonslist, columns=headings)
df1.head()

,season,number,name,description,url
0,1,1,Simpsons Roasting on an Open Fire,Homer decides to gamble on a,http://www.simpsonsworld.com/video/273376835817
1,1,2,Bart The Genius,Bart is believed to be a genius after he switc...,http://www.simpsonsworld.com/video/283744835990
2,1,3,Homer's Odyssey,"Fired from his job at the Nuclear Power Plant,...",http://www.simpsonsworld.com/video/273381443699
3,1,4,There's No Disgrace Like Home,"After attending the annual company picnic, Hom...",http://www.simpsonsworld.com/video/273392195780
4,1,5,Bart the General,Grandpa Simpson aids Bart in his war against a...,http://www.simpsonsworld.com/video/300934723994


In [6]:
print('Total number of episodes: ' + str(df1.shape[0]))

Total number of episodes: 644


## Get episode script

For each episode, we want to open the corresponding link and extract the episode script.

### Test for one episode

`url = df1['URL'][0]
try:
    page = urllib.request.urlopen(url).read()
except:
    print("Cannot read url")
soup = bs4.BeautifulSoup(page, 'html.parser')
script = [line.p.get_text() for line in soup.find_all('div', attrs={'class': 'script-message'})]`

The above code ran correctly, so now we can loop over many episodes.

In [7]:
# time how long it takes to get x number of scripts
start = time.time()

# loop over all episodes with links in dataframe

n_ep = 644

ep_scripts = []
for url in df1['url'][0:n_ep+1]:
    
    # read page html into string
    try:
        page = urllib.request.urlopen(url).read()
    except:
        print("Cannot read url")

    # create soup object
    soup = bs4.BeautifulSoup(page, 'html.parser')
    
    # get script
    script = [line.p.get_text() for line in soup.find_all('div', attrs={'class': 'script-message'})]
    ii = 1
    
    for line in script:
        ep_scripts.append([url, ii, line])
        ii+=1

# set headings
headings = ['url', 'linenumber', 'text']

# print out timer results
end = time.time()
print('Time elapsed: ' + str(round(end - start, 4)) + ' seconds')

print('Number of episodes: ' + str(n_ep))

Time elapsed: 878.265 seconds
Number of episodes: 644


In [8]:
# create pandas dataframe
df2 = pd.DataFrame(ep_scripts, columns=headings)
df2.head()

,url,linenumber,text
0,http://www.simpsonsworld.com/video/273376835817,1,(Street: ext. street - establishing - night)
1,http://www.simpsonsworld.com/video/273376835817,2,(Car: int. car - night)
2,http://www.simpsonsworld.com/video/273376835817,3,"Marge Simpson: Ooo, careful, Homer."
3,http://www.simpsonsworld.com/video/273376835817,4,Homer Simpson: There's no time to be careful.
4,http://www.simpsonsworld.com/video/273376835817,5,Homer Simpson: We're late.


## SQL queries in pandas

Let's learn how to write a SQL query with pandas. [This page](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html) is a good resource for SQL-pandas conversions. Below is the SQL query we want to convert.

`SELECT text FROM df2  
  JOIN df ON df.url = df2.url  
  WHERE df.season = 1 AND df.episode = 2  
  ORDER BY linenumber`

In [9]:
# inner join
#df3 = pd.merge(df, df2, on='URL')
# where
#df3[df3['Season'] == '1']['Text']
# and where
#df3[(df3['Season'] == '1') & (df3['Number'] == '2')]['Text']
# order by
#df3[(df3['Season'] == '1') & (df3['Number'] == '2')].sort_values('LineNumber')['Text']

## Put data into PostgreSQL database

We'll first need to create a PostgreSQL database through Python.

This can be done by entering the following into a terminal window: `createdb simpsonsscripts -U hsf001`

In [10]:
# create database name
dbname = 'simpsonsscripts'
username = 'hsf001'

# connect to postgresl
import psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [11]:
from sqlalchemy import create_engine

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

# put dataframe into sql
# first argument is name of table
df1.to_sql('episodes', engine, if_exists='replace')
df2.to_sql('scripts', engine, if_exists='replace')

postgres://hsf001@localhost/simpsonsscripts


In [29]:
# database analysis - episodes and scripts

# sql query
sql_query = """
SELECT name FROM episodes WHERE season='10';
"""

# should be 644 episodes

sql_query = """
SELECT * FROM scripts where linenumber=1;
"""

# only has linenumber=1 for 564 scripts

sql_query = """
SELECT season, number, name, episodes.url, text FROM episodes  
  LEFT JOIN scripts ON episodes.url = scripts.url WHERE scripts.linenumber IS NULL
  ORDER BY episodes.season, episodes.number
"""

# missing 80 episode scripts... 2 from season 20, a few others (21, 25, 26), all from seasons 27+

test_from_sql = pd.read_sql_query(sql_query,con)
test_from_sql

# The URLs of these shows that are missings scripts in the database don't have the scripts online.

,season,number,name,url,text
0,20,21,Coming To Homerica,http://www.simpsonsworld.com/video/729848899646,None
1,20,4,Treehouse of Horror XIX,http://www.simpsonsworld.com/video/691895363987,None
2,21,6,Pranks and Greens,http://www.simpsonsworld.com/video/792414787985,None
3,25,20,Brick Like Me,http://www.simpsonsworld.com/video/311243331763,None
4,25,25,The Longest Daycare,http://www.simpsonsworld.com/video/588949571786,None
5,26,17,Waiting for Duffman,http://www.simpsonsworld.com/video/420981315819,None
6,26,18,Peeping Mom,http://www.simpsonsworld.com/video/430426691868,None
7,26,19,The Kids Are All Fight,http://www.simpsonsworld.com/video/434682947703,None
8,26,20,Let's Go Fly A Coot,http://www.simpsonsworld.com/video/438408259807,None
9,26,21,Bull-E,http://www.simpsonsworld.com/video/442879555692,None


### Analysis of words spoken by main characters by season

season1
season2

plot something